In [1]:
import pandas as pd
import numpy as np
import os, random, csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import scipy.sparse as ss
# https://github.com/vividvilla/csvtotable
def clean_members(x,mdict):
    judges = []
    for item in x.split("_"):
        item = item.strip()
        if item in list(mdict.keys()):
            judges.append(str(mdict[item]))
        else:
            continue
    judges = '_'.join(list(set(judges)))
    return judges 

def clean_sections(x):
    x = str(x)
    if len(x) <100:
        newc = x
    else:
        newc=''
    return newc

def map_vectwords(x,words):
    pwords = []
    for w in words:
        if w in x:
            pwords.append(w)
        else:
            continue
    pwords = sorted(list(set(pwords)))
    return ', '.join(pwords)

In [2]:
mainpath = str(os.getcwd())+'/'

df = pd.read_csv(mainpath+"output/6_case_db.csv",sep='\t')
df['datetime'] = pd.to_datetime(df['date'])
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['month_year'] = df['month'].astype(str) + '_' + df['year'].astype(str)
df['quarter'] = pd.PeriodIndex(df.datetime, freq='Q')
df['numone'] = 1
df.fillna(np.nan,inplace=True)
df = df.sample(frac=1).reset_index(drop=True)

df['id'] = list(range(0,df.shape[0]))

members = [x for x in list(set([x.strip() for x in '_'.join(list(set(df['member'].tolist()))).split('_')])) if 'ottawa' not in x.lower()]
random.shuffle(members)
mdict = dict(zip(members,list(range(0,len(members)))))
print(len(mdict))
df['cleanmember'] = df['member'].apply(lambda x:clean_members(x,mdict))
df['csections'] = df['cleansections'].apply(lambda x: clean_sections(x))

docs = df['topiccleantext'].tolist()
vectorizern = TfidfVectorizer(ngram_range=(1, 3), max_features = 6000, max_df = 0.9, min_df = 0.05)
dtm_tf = vectorizern.fit_transform(docs)
dtm_tf = ss.csr_matrix(dtm_tf)
words = vectorizern.get_feature_names_out()

df['keywords'] = df['topiccleantext'].apply(lambda x: map_vectwords(x,words))
# df.fillna('',inplace=True)

163


In [3]:
pdf = df[['id','date','newloc','cleanmember','csections','applicant','winner','keywords']]
pdf.columns = ['Case Id','Date','Office Location','Adjudicators','Act Sections','Applicant','Winner','Keywords']

In [4]:
pdf

,Case Id,Date,Office Location,Adjudicators,Act Sections,Applicant,Winner,Keywords
0,0,2012-07-16,SouthWest_London,85,31,1,0,"12, 13, 29, 31, 31 rtaa, accord, accordance, a..."
1,1,2017-10-13,TOSouth,53,30 31,1,0,"13, 14, 15, 31, 31 rtaa, 78, act, action, addi..."
2,2,2017-12-19,TOEast,60,871 130 30,0,0,"12, 13, 14, 15, 29, 31, abatement, account, ac..."
3,3,2017-12-04,Central_Mississauga,139,nan,1,1,"12, 13, 14, 15, 31, access, accord, acknowledg..."
4,4,2009-09-03,SouthWest_London,108,135,1,0,"13, 31, act, advise, agree, amount, applicatio..."
...,...,...,...,...,...,...,...,...
36456,36456,2020-10-13,Northern_Sudbury,33,69,0,0,"12, 13, 14, 31, 31 arrears, 31 arrears serve, ..."
36457,36457,2011-01-19,Central_Mississauga,88,69,0,1,"69, 69 rtaa, 69 rtaa apply, also, application ..."
36458,36458,2017-01-03,TOEast,108,30 31 69,1,1,"12, 13, 14, 15, 29, 31, 31 arrears, 69, 69 rta..."
36459,36459,2021-08-03,Eastern_Ottawa,108,69,0,0,"14, 15, 31, 31 arrears, 31 arrears serve, 69, ..."


In [5]:
pdf.to_csv(mainpath + 'output/8_public_onltb_db.csv',sep=',',index=False)